# Trabalho Unidade 03 - Análise de hashtags relacionadas a criptomoedas no Twitter

Autores:

* Darlan de Castro Silva Filho - 20200000607;
* Marcos Henrique Fernandes Marcone - 20200000760;

Disciplina: IMD1155 - ANÁLISE DE REDES - T01 (2021.2)

Professor: Ivanovitch Medeiros Dantas da Silva

## Análise de hashtags relacionadas a criptomoedas no Twitter


O trabalho tem como objetivo a análise das hashtags mais comumente encontradas no Twitter relacionadas ao mundo das criptomoedas.

Para isso ser feito foram escolhidas as 10 criptomoedas com o maior valor de mercado no dia **09 de Fevereiro de 2022**. Para isso foi utilizado o site [Coin Market Cap](https://coinmarketcap.com/).

Assim as 180 requisições que são disponibilizadas pela API do Twitter, no intervalo de 15 minutos, foram divididas na busca de 10 *hashtags*. Cada criptomoeda foi representada com a sigla como chave de pesquisa. 

Ao todo, ao longo de mais de uma semana foram obtidos mais de 127000 tweets formando uma base dados bem sólida. Dessa forma, foi possível analisar quais são os tópicos que mais estão ligados quando o assunto é criptomoedas no Twitter.

Para acessar a segunda parte deste trabalho, que é o grafo obtido através dos dados deste notebook, basta clicar no seguinte link: [link](https://drive.google.com/file/d/1xTMFlQgPWho3B9NzvDndZJ4jmmYRpUXV/view?usp=sharing).

In [ ]:
# Instalação da biblioteca Twython para manipulação da API do Twitter
!pip install Twython

In [ ]:
# Uso de arquivo externos ao notebook no google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download do dataset
!gdown https://drive.google.com/uc?id=1-4BIILGxf68lSC-yQ-Eel_LjEB3DnlzT
PATH_DATAFRAME = '/dataframeTemporario.csv'

In [ ]:
# Caminho para dataset de tweets recuperados
PATH_DATAFRAME = '/content/drive/MyDrive/2021.2/Análise de Redes/Trabalho 3 Unidade/dataframeTemporario.csv'

In [ ]:
# Importando as principais bibliotecas
from twython import Twython, TwythonError
import pandas as pd
import itertools
import time
import plotly.express as px

## Autenticação com API do Twitter

Para a autenticação com o Twitter são utilizadas as chaves de API fornecidas para contas que foram validadas para uso direto com a sua API.

In [ ]:
# Chaves de autenticação com a API do Twitter - Arquivo keys.txt
# Abre o arquivo keys.txt
my_file = open("keys.txt", "r")

# Ler todo o conteúdo em um única string
content = my_file.read()

# separa as linhas com o carectere \n como separador
API_KEY, API_SECRET_KEY = content.split("\n")

# Fecha o arquivo
my_file.close()

### Entrando com as informação e obtendo uma URL de autorização

In [ ]:
# Obtendo token de verificação ao enviar as chaves para a API do Twitter
twitter = Twython(API_KEY, API_SECRET_KEY)

tokensDeAutenticacao = twitter.get_authentication_tokens()
print(tokensDeAutenticacao['auth_url'])

https://api.twitter.com/oauth/authenticate?oauth_token=UHo4OwAAAAABYzBYAAABfwos24c


### Autorizando a utilização do app com o PIN de verificação 

In [ ]:
# Validação do PIN recebido na etapa anterior e recebimento dos tokens de acesso
VERIFIER = '6176399' #Inserir PIN aqui

twitter = Twython(API_KEY, API_SECRET_KEY,
                  tokensDeAutenticacao['oauth_token'],
                  tokensDeAutenticacao['oauth_token_secret'])

tokensAutorizados = twitter.get_authorized_tokens(VERIFIER)

### Utilizando os Tokens de autorização

In [ ]:
# Utilização dos tokens recebidos para liberação do acesso
twitter = Twython(API_KEY, API_SECRET_KEY,
                  tokensAutorizados['oauth_token'],
                  tokensAutorizados['oauth_token_secret'])

twitter.verify_credentials()

{'contributors_enabled': False,
 'created_at': 'Wed Feb 02 17:47:26 +0000 2022',
 'default_profile': True,
 'default_profile_image': False,
 'description': '',
 'entities': {'description': {'urls': []}},
 'favourites_count': 0,
 'follow_request_sent': False,
 'followers_count': 0,
 'following': False,
 'friends_count': 1,
 'geo_enabled': False,
 'has_extended_profile': True,
 'id': 1488931987704397824,
 'id_str': '1488931987704397824',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': None,
 'listed_count': 0,
 'location': '',
 'name': 'Darlan de Castro',
 'needs_phone_verification': False,
 'notifications': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': None,
 'profile_background_image_url_https': None,
 'profile_background_tile': False,
 'profile_image_url': 'http://pbs.twimg.com/profile_images/1488932082927783941/Yp9Yy15A_normal.png',
 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1488932082927783941/Yp9Yy15A_norma

## Análise Expoloratória dos dados

Esta seção apresenta uma Análise Exploratória dos Dados (EDA), que consiste na composição de um dataframe pandas com base nos tweets que foram obtidas através da API do Twitter. 

### Formato do tweet que será salvo no dataset 

* **user_screen_name**: Identificador textual único do usuário ("@");
* **user_location**: Localização do usuário;
* **user_name**: Nome do usuário;
* **favorite_count**: Quantidade de favoritos do tweet;
* **created_at**: Quando o tweet foi criado;
* **retweet_count**: Quantidade de retweets do tweet;
* **id**: Identificador numérico único do tweet;
* **hashtags**: Lista com as hashtags mencionadas no tweet.

###Filtragem

É preciso ressaltar que para formar o dataframe foram aplicados os seguintes filtros: eliminação dos tweets que são **truncados**, como também foram descartados aqueles que retornaram o campo ***hashtags* vazio**.

### Montando o Dataframe

Inicialmente, como a aplicação foi executada em diferentes dias, então carrega-se o dataframe obtido até o dia anterior.

In [ ]:
# Carregando o dataset para a adição de novos tweets
velhoDF = pd.read_csv(PATH_DATAFRAME)

#velhoDF = pd.DataFrame()

In [ ]:
# Dimensões do dataset 
velhoDF.shape

(125257, 8)

In [ ]:
# Exemplos de tweets já salvos
velhoDF.head()

,id,user_screen_name,user_location,user_name,favorite_count,retweet_count,hashtags,created_at
0,1.491879e+18,magolito98,NaN,Gemfai 💎,0.0,1.0,"['dogearmy', 'Elonmusk', 'dogecoin']",Thu Feb 10 20:56:57 +0000 2022
1,1.491879e+18,los_aytises,NaN,de_campo,0.0,45.0,['BTC'],Thu Feb 10 20:56:54 +0000 2022
2,1.491879e+18,rCryptoReddit,Online,r/CryptoCurrency,0.0,0.0,"['bitcoin', 'btc', 'cryptocurrency', 'crypto',...",Thu Feb 10 20:56:52 +0000 2022
3,1.491879e+18,KubatRustu,NaN,Rüştü Kubat,0.0,80.0,"['Metaverse', 'Mogul']",Thu Feb 10 20:56:50 +0000 2022
4,1.491879e+18,KetenNimet,NaN,Nimet Keten,0.0,38.0,"['agesa', 'akyho', 'alka', 'anhyt', 'arase', '...",Thu Feb 10 20:56:47 +0000 2022


In [ ]:
# Função de formatação do tweet antes de salvá-lo no dataset
def formatarTweet(tweet):
  tweetFormatado = {}
  tweetFormatado['user_screen_name'] = tweet['user']['screen_name']
  tweetFormatado['user_location'] = tweet['user']['location']
  tweetFormatado['user_name'] = tweet['user']['name']
  tweetFormatado['favorite_count'] = tweet['favorite_count']
  tweetFormatado['created_at'] = tweet['created_at']
  tweetFormatado['retweet_count'] = tweet['retweet_count']
  tweetFormatado['id'] = tweet['id']
  tweetFormatado['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
  return tweetFormatado
  

As hashtags foram escolhidas com base no preço de mercado do dia **09 de Feveireiro de 2022**. Foram escolhidas as 10 mais valiosas.

In [ ]:
# Definição das hashtags que serão buscadas e passo atual do iterador
queries = ['#BTC', '#ETH', '#BNB', '#USDC', '#XRP', '#ADA', '#SOL', '#LUNA', '#AVAX', '#USDT']
passoAtual = 1

Busca pelos tweets utilizando a API do Twitter e a biblioteca **Twython**.

In [ ]:
# Busca das hashtags e inserção de tweets no dataset
tweetsProcurados = []
NUMERO_DE_TWEETS_PARA_BUSCA = 1300
try:
  for query in queries:
    print('Procurando: ', query)
    cursor = twitter.cursor(twitter.search, q=query, count=100, result_type='mixed')
    tweetsProcurados += list(itertools.islice(cursor, NUMERO_DE_TWEETS_PARA_BUSCA * (passoAtual - 1), NUMERO_DE_TWEETS_PARA_BUSCA * passoAtual))
    print('Quantidade de buscas restante: ', twitter.get_application_rate_limit_status()['resources']["search"]['/search/tweets']['remaining'])
except Exception as e:
  print(e)
finally:
  print('Quantidade de resultados: ', len(tweetsProcurados))
  dict_ = {'id': [], 'user_screen_name': [], 'user_location': [], 'user_name': [], 'favorite_count': [], 'retweet_count': [], 'hashtags': [], 'created_at': []}
  for tweet in tweetsProcurados:
    # Descarta tweets com o conteúdo textual truncado
    if tweet['truncated']:
      continue
    tweetFormatado = formatarTweet(tweet)
    # Descarta tweets com a lista de hashtags vazia
    if tweetFormatado['hashtags'] == []:
      continue
    for key in tweetFormatado:
      dict_[key].append(tweetFormatado[key])
  df = pd.DataFrame(dict_)
  print('Dimensoes df:', df.shape)
  frames = [velhoDF, df]
  novoDF = pd.concat(frames)
  novoDF.drop_duplicates(subset=['id'], inplace=True)
  print('Dimensoes novoDF:', novoDF.shape)
  print('Quantidade de novos tweets:', novoDF.shape[0] - velhoDF.shape[0])
  novoDF.to_csv(PATH_DATAFRAME, index=False)
  velhoDF = novoDF
  passoAtual += 1
  time.sleep(900)

In [ ]:
# Verificação da quantidade de requisições restantes no limite de 15 minutos
twitter.get_application_rate_limit_status()['resources']["search"]

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1645124800}}

### Dados

Com o dataframe devidamente formado, pode-se obter alguns dados interessantes, como por exemplo, aqueles que possuem o maior número de likes.

In [ ]:
ordenacaoPorNumeroFavoritos = novoDF.sort_values(by=['favorite_count'], ascending=False)
ordenacaoPorNumeroFavoritos.head()

,id,user_screen_name,user_location,user_name,favorite_count,retweet_count,hashtags,created_at
2381,1.491779e+18,cz_binance,NaN,CZ 🔶 Binance,14542.0,2220.0,"['bitcoin', 'BNB']",Thu Feb 10 14:20:41 +0000 2022
3578,1.491169e+18,EleanorTerrett,"New York, USA",Eleanor Terrett,3568.0,532.0,"['XRP', 'Cardano']",Tue Feb 08 21:58:02 +0000 2022
26956,1.492164e+18,MartiniGuyYT,Bitcoin 🚀🌙,That Martini Guy ₿,3201.0,346.0,"['BITCOIN', 'BTC']",Fri Feb 11 15:51:08 +0000 2022
14475,1.491601e+18,reporterchris,Toronto,Chris Johnston,2514.0,119.0,['BTC'],Thu Feb 10 02:31:24 +0000 2022
3836,1.491166e+18,RippleXrpie,NaN,JackTheRippler🤫🐻🏦,2213.0,136.0,['XRP'],Tue Feb 08 21:44:11 +0000 2022


Outra métrica interessante é verificar quais tweets possuem o maior número de retweets.

In [ ]:
ordenacaoPorNumeroRetweet = novoDF.sort_values(by=['retweet_count'], ascending=False)
ordenacaoPorNumeroRetweet.head()

,id,user_screen_name,user_location,user_name,favorite_count,retweet_count,hashtags,created_at
2829,1.491572e+18,Z3rou,Brasil,Zirou,0.0,38052.0,['BNB'],Thu Feb 10 00:35:40 +0000 2022
2384,1.492325e+18,Rakeshp15735056,Kolkata,summpk,0.0,35215.0,"[Airdrops, Giveaways]",Sat Feb 12 02:29:32 +0000 2022
2111,1.493019e+18,Melissa53702410,,Melissa Taylor,0.0,35213.0,"[Airdrops, Giveaways]",Mon Feb 14 00:26:17 +0000 2022
43551,1.492956e+18,komoriri1,NaN,komoriri,0.0,35191.0,"['Airdrops', 'Giveaways']",Sun Feb 13 20:16:07 +0000 2022
35397,1.492928e+18,dnhung851,Việt Nam,Nhung Do,0.0,35176.0,"['Airdrops', 'Giveaways']",Sun Feb 13 18:25:56 +0000 2022
